In [ ]:
 from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import IPython.display as ipd
from IPython.display import Audio
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM,BatchNormalization , GRU
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
!apt-get update
!apt-get install -y libsndfile1

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [52.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,755 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,370 kB]
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,077 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:13 http://archive.ubunt

In [ ]:
ravdess = '/content/drive/My Drive/DATASET_1'
files_lis_dir = os.listdir(ravdess)
print(files_lis_dir)
print("Done")


['Actor_24', 'Actor_14', 'Actor_19', 'Actor_17', 'Actor_16', 'Actor_18', 'Actor_20', 'Actor_22', 'Actor_21', 'Actor_23', 'Actor_15', 'Actor_13', 'Actor_06', 'Actor_09', 'Actor_08', 'Actor_10', 'Actor_11', 'Actor_05', 'Actor_04', 'Actor_12', 'Actor_07', 'Actor_03', 'Actor_01', 'Actor_02']
Done


In [ ]:
#for ravdess dataset, storing path and emotion
file_path = []
file_emotion = []
for f in files_lis_dir:
  actor = os.listdir(os.path.join(ravdess, f))
  for a in actor: #for all audio files
    audio = a.split('.')[0].split('-')
    file_emotion.append(int(audio[2]))
    file_path.append(ravdess + f + '/' + a)
print("Cell executed")
print("size of file path : ", len(file_path), "and file emotion: ", len(file_emotion))

Cell executed
size of file path :  1440 and file emotion:  1440


In [ ]:
# for i in range(24):
#   print(actor[i])
# for i in range( 10):
#   print(store_emotion[i])

# print("File path : ", store_path[1])

In [ ]:
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
path_df = pd.DataFrame(file_path, columns=['Path'])
ravdess_df = pd.concat([emotion_df, path_df], axis=1)
ravdess_df.Emotions.replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust',8:'surprise'},inplace=True)
print("cell exec")

cell exec


In [ ]:
print(ravdess_df.head())
print("---------------")
print(ravdess_df.tail())
print("----------")
print(ravdess_df['Emotions'].value_counts())


  Emotions                                               Path
0      sad  /content/drive/My Drive/DATASET_1Actor_24/03-0...
1    angry  /content/drive/My Drive/DATASET_1Actor_24/03-0...
2    happy  /content/drive/My Drive/DATASET_1Actor_24/03-0...
3     fear  /content/drive/My Drive/DATASET_1Actor_24/03-0...
4     fear  /content/drive/My Drive/DATASET_1Actor_24/03-0...
---------------
      Emotions                                               Path
1435   disgust  /content/drive/My Drive/DATASET_1Actor_02/03-0...
1436  surprise  /content/drive/My Drive/DATASET_1Actor_02/03-0...
1437  surprise  /content/drive/My Drive/DATASET_1Actor_02/03-0...
1438  surprise  /content/drive/My Drive/DATASET_1Actor_02/03-0...
1439   disgust  /content/drive/My Drive/DATASET_1Actor_02/03-0...
----------
Emotions
neutral     288
sad         192
angry       192
happy       192
fear        192
disgust     192
surprise    192
Name: count, dtype: int64


In [ ]:
#second dataset
crema ='/content/drive/My Drive/DATASET_2_CREMA'
# files = os.listdir(crema)
# num_files = len(files)
# print(num_files)

In [ ]:
crema_dir_list = os.listdir(crema)
file_path = []
file_emotion = []

for i in crema_dir_list:
    # storing file paths
    file_path.append(crema + i)
    # storing file emotions
    part=i.split('_')
    if part[2] == 'SAD':
        file_emotion.append('sad')
    elif part[2] == 'ANG':
        file_emotion.append('angry')
    elif part[2] == 'DIS':
        file_emotion.append('disgust')
    elif part[2] == 'FEA':
        file_emotion.append('fear')
    elif part[2] == 'HAP':
        file_emotion.append('happy')
    elif part[2] == 'NEU':
        file_emotion.append('neutral')
    else:
        file_emotion.append('Unknown')

emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
path_df = pd.DataFrame(file_path, columns=['Path'])
crema_df = pd.concat([emotion_df, path_df], axis=1)

crema_df.head()
print(crema_df.Emotions.value_counts())
print("Cell executed")
print("size of file path : ", len(file_path), "and file emotion: ", len(file_emotion))

Emotions
fear       1188
disgust    1187
sad        1187
angry      1187
happy      1186
neutral    1015
Name: count, dtype: int64
Cell executed
size of file path :  6950 and file emotion:  6950


In [ ]:
#third dataset
savee ='/content/drive/My Drive/DATASET_3_SAVEE/ALL'
savee_dl = os.listdir(savee)



In [ ]:
# files = os.listdir(savee)
# for entry in files:

#     entry_path = os.path.join(savee, entry)
#     if os.path.isdir(entry_path):
#         subfiles = os.listdir(entry_path)
#         print(f"Number of files in '{entry}': {len(subfiles)}")
# print("done")

In [ ]:
file_path = []
file_emotion = []
for i in savee_dl:
    file_path.append(savee + i)
    part = i.split('_')[1]
    ele = part[:-6]
    if ele=='a':
        file_emotion.append('angry')
    elif ele=='d':
        file_emotion.append('disgust')
    elif ele=='f':
        file_emotion.append('fear')
    elif ele=='h':
        file_emotion.append('happy')
    elif ele=='n':
        file_emotion.append('neutral')
    elif ele=='sa':
        file_emotion.append('sad')
    else:
        file_emotion.append('surprise')


emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
path_df = pd.DataFrame(file_path, columns=['Path'])
savee_df = pd.concat([emotion_df, path_df], axis=1)
savee_df.head()
print(savee_df['Emotions'].value_counts())
print("Cell executed")
print("size of file path : ", len(file_path), "and file emotion: ", len(file_emotion))

Emotions
neutral     120
angry        60
fear         60
happy        60
disgust      60
sad          60
surprise     60
Name: count, dtype: int64
Cell executed
size of file path :  480 and file emotion:  480


In [ ]:
#fourth dataset
tess = '/content/drive/My Drive/tess_moved'
tess_dl = os.listdir(tess)

In [ ]:
file_emotion = []
file_path = []
for dir in tess_dl:
    files = os.listdir(os.path.join(tess, dir))
    for file in files:
        part = file.split('.')[0]
        part = part.split('_')[2]
        if part == 'ps':
            file_emotion.append('surprise')
        else:
            file_emotion.append(part)
        full_path = os.path.join(tess, dir, file)
        file_path.append(full_path)

emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
path_df = pd.DataFrame(file_path, columns=['Path'])
tess_df = pd.concat([emotion_df, path_df], axis=1)
print(tess_df.head())
print(tess_df.Emotions.value_counts())
print("Cell executed")
print("size of file path : ", len(file_path), "and file emotion: ", len(file_emotion))


In [ ]:
#combining all
df = pd.concat([ravdess_df, crema_df, tess_df, savee_df], axis = 0)
df.to_csv("df.csv",index=False)
print(df.head())
print(df['Emotions'].value_counts())

In [ ]:
df.to_csv("/content/drive/My Drive/df.csv", index=False)


In [ ]:
# #plotting one sample

# plt.figure(figsize = (10, 5))
# spect = librosa.feature.melspectrogram(y = data, sr= sr, n_mels = 128, fmax = 10000)
# log_spect = librosa.power_to_db(spect) #now in db, log le liya
# librosa.display.specshow(log_spect, y_axis = 'mel', sr = sr, x_axis = 'time')
# plt.title('Mel Spectrogram ')
# plt.colorbar(format='%+2.0f dB')



In [ ]:
path = '/content/drive/My Drive/df.csv'
df1 = pd.read_csv(path)

In [ ]:
df.head()

#features


In [ ]:
def oneliner_ae(signal, frame_size, hop_length):
  return np.array([max(signal[i:i+frame_size]) for i in range(0,len(signal), hop_length)])

def rms_energy(signal, frame_size, hop_length):
  return np.array(librosa.feature.rms(y = signal, frame_length = frame_size, hop_length = hop_length))[0] #0th row to be accessed

def zcr(signal, frame_size, hop_length):
  return np.array(librosa.feature.zero_crossing_rate(y = signal, frame_length = frame_size, hop_length = hop_length))[0]

In [ ]:
def calc_split_freq_bin(spectrogram, split_freq, sr):
  freq_range = sr/2
  freq_delta_per_bin = freq_range / spectrogram.shape[0] #.[0] for time accessing
  split_freq_bin = np.floor(freq_range/freq_delta_per_bin)
  return int(split_freq_bin)

def band_energy_ratio(spectrogram, split_freq, sr):
  split_freq_bin = calc_split_freq_bin(spectrogram, split_freq, sr)

  ber = []
  power_spec = np.abs(spectrogram)**2
  power_spec = power_spec.T

  for i in power_spec:
    power_low_freq = np.sum(i[:split_freq_bin])
    power_high_freq = np.sum(i[split_freq_bin:])
    ber_current = power_low_freq / power_high_freq
    ber.append(ber_current)
  return np.array(ber)


In [ ]:
def spectral_centroid(signal, sr, frame_size, hop_length):
  return librosa.fratures.spectral_centroid(signal, sr, n_fft = frame_size, hop_length = hop_length)[0]

def bandwidth(signal, sr, frame_size, hop_length):
  return librosa.fratures.bandwidth(signal, sr, n_fft = frame_size, hop_length = hop_length)[0]

In [ ]:
def mfcc(signal, coef, sr):
  mfcc1 = librosa.feature.mfcc(signal, n_mfcc = coef, sr = sr)
  mfcc_delta_1 = librosa.feature.delta(mfcc1)
  mfcc_delta_2 = librosa.feature.delta(mfcc1, order = 2)
  final = np.concatenate((mfcc1, mfcc_delta_1, mfcc_delta_2))
  return final

In [ ]:
debussy_aud = '/content/debussy.wav'
duke_aud = '/content/duke.wav'
redhot_aud = '/content/redhot.wav'
violin_c_aud = '/content/violin_c.wav'
piano_aud = '/content/piano.wav'

In [ ]:
debussy , sr = librosa.load(debussy_aud)
duke , _ = librosa.load(duke_aud)
debussy , _ = librosa.load(redhot_aud)
violin_c, _  = librosa.load(violin_c_aud)
piano, _  = librosa.load(piano_aud)

In [ ]:
def plot_mag_spec(signal, sr, f_ratio): #fratio for the fraction
  ft = np.fft.fft(signal)
  mag_spec = np.abs(ft)
  plt.figure(figsize = (20, 6))
  freq = np.linspace(0, sr, len(mag_spec))
  freq_bins = int(len(freq)*f_ratio)
  plt.plot(freq[:freq_bins], mag_spec[:freq_bins])
  plt.title("magnitude spectrum")
  plt.xlabel("Freq (Hz)")
  plt.ylabel("Magnitude")
  plt.show()

def vis_spectrogram(Y, sr, hop_length, y_axis = "linear"):
  plt.figure(figsize = (20, 8))
  librosa.display.specshow(
                         Y,
                        sr = sr,
                         hop_length = hop_length,
                         x_axis = "time",
                         y_axis = y_axis

  )
  plt.colorbar(format = "%+2.f")


In [ ]:
# plot_mag_spec(piano, sr, 1)
#spectrograms
#s_scale = librosa.stft(signal, n_fft, hop_size )
#y_scale = np.abs(s_scale) ** 2 # complex -> real mag
s_scale = librosa.stft(piano, n_fft = 1024, hop_length = 512)
y_scale = np.abs(s_scale) ** 2
vis_spectrogram(y_scale, sr, 512)
y_log_scale = librosa.power_to_db(y_scale)
#so now, log amplitude spectrogram if we pass this
vis_spectrogram(y_log_scale, sr, 512)
#now log amp log freq, so log spectrrogram, as we perceivee aisa dikhega
vis_spectrogram(y_log_scale, sr, 512, y_axis = "log")

In [ ]:
#mel spectrograms
filter_banks = librosa.filters.mel(n_fft = 2048, sr = 22050, n_mels =10)
filter_banks.shape
mel_spectrogram = librosa.feature.melspectrogram(y = piano, sr = sr, n_fft =2048, hop_length = 512, n_mels = 10)
# mel_spectrogram.shape
log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)

In [ ]:
plt.figure(figsize = (20, 8))
librosa.display.specshow(
    log_mel_spectrogram,
    x_axis = "time",
    y_axis = "mel",
    sr = sr
)
plt.colorbar(format = "%+2.f")
plt.show()
# can see 10 mels as n_mels is 10 here

In [ ]:
debussy_spec = librosa.stft(debussy, n_fft = 2048, hop_length = 512)
debussy_ber  = band_energy_ratio(debussy_spec, 2000, 22050)
# debussy_ber.shape

In [ ]:
#visualize spec:
frames = range(len(debussy_ber))
t = librosa.frames_to_time(frames)
plt.figure(figsize = (20, 8))
plt.plot(t, debussy_ber, color= 'b')
plt.show()